In [1]:
!pip install duckdb

import duckdb
import pandas as pd
import itertools

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
#con = duckdb.connect(database='/content/drive/MyDrive/projecteADSDB/formatted/formatted.duckdb', read_only=True)
con = duckdb.connect(database='./formatted/formatted.duckdb', read_only=True)
print(con.execute('SELECT 1').fetchall())

[(1,)]


We assume data integration can be done by loading the latest table of each data source based on the timestamp

In [4]:
l = con.execute("SHOW TABLES").fetchall()
out = list([t for (t,) in l])

In [5]:
out.sort()

def get_field_sub(x): return x.split('$')[0]

mylist = sorted(out, key=get_field_sub)
data = dict((x.split('$')[0],list(y)) for x, y in itertools.groupby(out, get_field_sub))

In [6]:
dsdict = {}

for ds in data:
  dsdict[ds] = pd.DataFrame()
  for dsversion in data[ds]:
    newdf = con.execute(f'SELECT * FROM {dsversion}').df()
    dsdict[ds] = pd.concat([dsdict[ds],newdf]).drop_duplicates().reset_index(drop=True)
    

Store data to trusted

In [7]:
#con = duckdb.connect(database='/content/drive/MyDrive/projecteADSDB/trusted/trusted.duckdb', read_only=False)
con = duckdb.connect(database='./trusted/trusted.duckdb', read_only=False)

In [8]:
for ds in dsdict:
  df = dsdict[ds]
  con.execute(f'DROP TABLE IF EXISTS {ds}')
  con.execute(f"CREATE TABLE IF NOT EXISTS {ds} AS SELECT * FROM df")


In [9]:
con.execute("SHOW TABLES").fetchall()

[('Country_Level_Data',), ('Governance_Data',), ('Olympics_Data',)]

In [10]:
con.close()